In [6]:
#### 0. 페이지에서 레스토랑 정보를 모두 조회

import requests

restaurants = [] # 배열 선언
for page in range(0, 2): # 0부터 576까지 있음 시간 간격두고 크롤링 추가 필요 => range(0, 577)
    url = f"https://www.bluer.co.kr/api/v1/restaurants?page={page}&size=30&query=&foodType=&foodTypeDetail=&feature=&location=&locationDetail=&area=&areaDetail=&priceRange=&ribbonType=&recommended=false&isSearchName=false&tabMode=single&searchMode=ribbonType&zone1=&zone2=&zone2Lat=&zone2Lng="

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Accept": "application/hal+json",
        "x-requested-with": "XMLHttpRequest"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json() 
        restaurants.append(data["_embedded"]["restaurants"])

In [7]:
print(data["_embedded"]["restaurants"][0])  # 첫번째 레스토랑 데이터를 조회하여 정보 확안

{'createdDate': 1231834010000, 'id': 4649, 'headerInfo': {'nameKR': '스시조', 'nameEN': 'Sushi Cho', 'nameCN': '', 'nickname': '', 'year': '2025', 'bookYear': '2025', 'ribbonType': 'RIBBON_THREE', 'ribbonTypeByOrdinal': 0}, 'defaultInfo': {'chefName': '', 'website': '', 'websiteInstagram': '', 'websiteFacebook': '', 'phone': '02-317-0373', 'openHours': '', 'closeHours': '', 'openHoursWeekend': '', 'closeHoursWeekend': '', 'dayOff': '연중무휴', 'app2Yn': False}, 'statusInfo': {'storeType': 'NOT', 'parking': '가능', 'creditCard': 'y', 'visit': '웨스틴조선호텔 20층', 'menu': '런치(Hall)(1인 15만5천원~20만5천원), 디너(Hall)(1인 19만4천원~33만원), 스시조회덮밥(10만원, 프리미엄 13만원), 복가라아게돌솥밥(12만원), 굴돌솥밥(7만8천원), 활새우튀김(11만원), 조리장특선모둠스시(14만5천원)', 'priceRange': '25만원 이상', 'openDate': '200810', 'businessHours': '12:00~15:00/17:30~22:00(마지막 주문 21:30) ', 'openEra': '', 'newOpenDate': '2008년 10월 '}, 'timeInfo': {'mondayByOpen': '', 'mondayByClose': '', 'tuesdayByOpen': '', 'tuesdayByClose': '', 'wednesdayByOpen': '', 'wednesdayByClose': '', '

In [8]:
#### 1. 전처리

import pandas as pd
from pandas import json_normalize

# DataFrame 변환
df = pd.DataFrame(data["_embedded"]["restaurants"])

# 필요 없는 columns 선별
columns_to_drop = ["createdDate", "id", "timeInfo", "gps", "tags", "status", "bookStatus",
                    "buzUsername", "business", "pageView", "brandMatchStatus", "brandRejectReason", 
                    "orderDescending", "foodTypeDetails", "countEvaluate", "bookmark", "features", 
                    "feature107", "brandBranches", "foodTypes", "brandHead", "firstImage", "firstLogoImage",
                    "_links"]

# 필요 없는 columns 삭제
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    ################## 중간산출물 : nested_restaurants.csv ####################
    # df.to_csv("nested_restaurants.csv", index=False)
    # print("CSV 파일 저장 완료.")
    ##########################################################################

# 내부 dictionary 형태의 정보를 Normalization
nested_columns = ["headerInfo", "defaultInfo", "statusInfo", "juso", "review", "etcInfo"]
    
# 각 열에 대해 정규화 실행
for column in nested_columns:
    n = pd.json_normalize(df[column])  # 정규화
    n.columns = [f"{column}_{subcol}" for subcol in n.columns]  # 열 이름 접두사 추가
    df = pd.concat([df.drop(columns=[column]), n], axis=1)  # 기존 열 삭제 후 결합

# 필요 없는 sub_columns 선별
sub_columns_to_drop = ["headerInfo_nickname", "headerInfo_year", "headerInfo_ribbonTypeByOrdinal",
                      "defaultInfo_websiteFacebook", "statusInfo_storeType", "statusInfo_openEra",
                      "statusInfo_newOpenDate", "juso_roadAddrPart2", "juso_jibunAddr", "juso_zipNo",
                      "juso_admCd", "juso_detBdNmList", "juso_zone2_1", "juso_zone2_2", "juso_map_1",
                      "juso_map_2", "review_readerReview", "review_businessReview", "review_editorReview",
                      "etcInfo_toilet", "etcInfo_toiletEtc", "etcInfo_chain", "etcInfo_close", "etcInfo_renewal",
                      "etcInfo_appYn", "etcInfo_projectNo", "etcInfo_reviewerRecommend", "etcInfo_onlySiteView",
                      "etcInfo_history", "etcInfo_mainMemo"]

# 필요 없는 sub_columns 삭제
df = df.drop(columns=[col for col in sub_columns_to_drop if col in df.columns])

# 결과 저장
df.to_csv("normalized_restaurants.csv", index=False)
print("CSV 파일 저장 완료!")

CSV 파일 저장 완료!


In [9]:
import numpy as np
import re

# 빈칸이나 "없음"으로 표기된 칸을 None으로 변경
df.replace(["", "없음"], None, inplace=True)

# foodDetailTypes의 리스트 해제
# 각 행의 값이 리스트라면 이를 문자열로 변환 (쉼표로 구분된 문자열로 병합)
if "foodDetailTypes" in df.columns:
    df["foodDetailTypes"] = df["foodDetailTypes"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

# ribbonType mapping
ribbon_mapping = {
    "RIBBON_ONE": 1,
    "RIBBON_TWO": 2,
    "RIBBON_THREE": 3,
    "RIBBON_NONE": 0
}

if "headerInfo_ribbonType" in df.columns:
    df["headerInfo_ribbonType"] = df["headerInfo_ribbonType"].map(ribbon_mapping)

# 웹사이트 열 통합
if "defaultInfo_website" in df.columns and "defaultInfo_websiteInstagram" in df.columns:
    def merge_websites(row):
        websites = [site for site in [row.get("defaultInfo_website"), row.get("defaultInfo_websiteInstagram")] if site]
        return ", ".join(websites) if websites else None

    df["defaultInfo_website_combined"] = df.apply(merge_websites, axis=1)
    df.drop(columns=["defaultInfo_website", "defaultInfo_websiteInstagram"], inplace=True)
    df.rename(columns={"defaultInfo_website_combined": "defaultInfo_website"}, inplace=True)

# statusInfo_openDate 형식 변환 (년도 4자리로 추출 후 "2024년" 형식으로 표기)
def extract_year_with_suffix(date):
    if isinstance(date, str):
        # 패턴 매칭으로 숫자 4자리 추출
        match = re.search(r'\d{4}', date)
        if match:
            return f"{int(match.group(0))}년"  # "2024년" 형식으로 반환
    return None

if "statusInfo_openDate" in df.columns:
    df["statusInfo_openDate"] = df["statusInfo_openDate"].apply(extract_year_with_suffix)

# 최종 결과를 CSV 파일로 저장
df.to_csv("cleaned_restaurants.csv", index=False)
print("수정된 최종 CSV 파일 저장 완료!")


수정된 최종 CSV 파일 저장 완료!
